# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine

import re
import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC

from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /home/jens/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jens/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jens/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jens/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_database.db')
df = pd.read_sql('messages', engine)
df.head()
X = df['message']
Y = df.drop(labels=['genre', 'id', 'message', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    #replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english')) 
    clean_tokens = []
 
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    filtered_tokens = [w for w in clean_tokens if not w in stop_words] 
        
    return filtered_tokens

for i in range(10):
    print(tokenize(df['message'][i]))
    print(df['message'][i])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Weather update - a cold front from Cuba that could pass over Haiti
['hurricane']
Is the Hurricane over or is it not over
['looking', 'someone', 'name']
Looking for someone but no name
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']
says: west side of Haiti, rest of the country today and tonight
['information', 'national', 'palace']
Information about the National Palace-
['storm', 'sacred', 'heart', 'jesus']
Storm at sacred heart of jesus
['please', 'need', 'tent', 'water', 'silo', 'thank']
Please, we need tents and water. We are in Silo, Thank you!
['would', 'like', 'receive', 'message', 'thank']
I would like to receive the messages, thank you
['croix', 'de', 'bouqu

In [79]:
tokenize("that's a test")

['thats', 'test']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [80]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [81]:
print(pipeline)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8d83c08620>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)


In [83]:
parameters = {
    'clf__estimator__n_estimators': 10,
    'clf__estimator__criterion': 'gini',
    'clf__estimator__max_features': ('auto'),#, 'log2'),
    'clf__estimator__min_samples_split': 2,
    'vect__ngram_range':  (1, 1),
    'tfidf__use_idf': False,
    'vect__max_df': 1.
}

pipeline.set_params(**parameters)

pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)


In [67]:
#Y_pred = cv.best_estimator_.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [84]:
#print(cv.best_estimator_.score(X_test, Y_test))
print(pipeline.score(X_test, Y_test))

0.2522918258212376


In [85]:
Y_test_array = Y_test.to_numpy().T

In [86]:
roc_score = []
for column in range(35):
    print(Y.columns[column])
    print(classification_report(Y_test_array[column], Y_pred.T[column]))
    roc = roc_auc_score(Y_test_array[column], Y_pred.T[column])
    print('roc auc score', roc)
    roc_score.append(roc)
    print(confusion_matrix(Y_test_array[column], Y_pred.T[column]))

related
              precision    recall  f1-score   support

       False       0.67      0.45      0.54      1246
        True       0.84      0.93      0.88      3990

    accuracy                           0.82      5236
   macro avg       0.76      0.69      0.71      5236
weighted avg       0.80      0.82      0.80      5236

roc auc score 0.6892300977162007
[[ 559  687]
 [ 280 3710]]
request
              precision    recall  f1-score   support

       False       0.88      0.98      0.93      4291
        True       0.79      0.42      0.55       945

    accuracy                           0.88      5236
   macro avg       0.84      0.70      0.74      5236
weighted avg       0.87      0.88      0.86      5236

roc auc score 0.6994053013628869
[[4186  105]
 [ 545  400]]
offer
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      5211
        True       0.00      0.00      0.00        25

    accuracy                           1

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       False       0.87      0.99      0.93      4539
        True       0.46      0.04      0.08       697

    accuracy                           0.87      5236
   macro avg       0.67      0.52      0.50      5236
weighted avg       0.82      0.87      0.81      5236

roc auc score 0.5176653286691492
[[4504   35]
 [ 667   30]]
infrastructure_related
              precision    recall  f1-score   support

       False       0.93      1.00      0.96      4881
        True       0.22      0.01      0.01       355

    accuracy                           0.93      5236
   macro avg       0.58      0.50      0.49      5236
weighted avg       0.88      0.93      0.90      5236

roc auc score 0.5020998352334866
[[4874    7]
 [ 353    2]]
transport
              precision    recall  f1-score   support

       False       0.96      1.00      0.98      4994
        True       0.58      0.07      0.13       242

    accuracy                 

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/si

In [87]:
print('average roc', np.mean(roc_score)) #0.83

average roc 0.5795234828564477


### 6. Improve your model
Use grid search to find better parameters. 

In [89]:

parameters = {
    'clf__estimator__n_estimators': [10],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_features': ['auto', 'sqrt'],#, 'log2'),
    'clf__estimator__min_samples_split': [2],#, 3),#,
    'vect__ngram_range': ((1, 1),(1,2)),#, (1, 2), (2,2)),
    'tfidf__use_idf': [True, False],
    'vect__max_df': [1.],
    #'vect__max_df': (0.1, 0.5, 1.0)
}

grid_search = GridSearchCV(pipeline, param_grid=parameters, n_jobs=6, scoring='roc_auc_ovo', verbose=1)
grid_fit = grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [97]:
grid_fit.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_features': 'auto',
 'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 10,
 'tfidf__use_idf': True,
 'vect__max_df': 1.0,
 'vect__ngram_range': (1, 1)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [91]:
Y_pred = grid_fit.best_estimator_.predict(X_test)


In [92]:
roc_score = []
for column in range(35):
    print(Y.columns[column])
    print(classification_report(Y_test_array[column], Y_pred.T[column]))
    roc = roc_auc_score(Y_test_array[column], Y_pred.T[column])
    print('roc auc score', roc)
    roc_score.append(roc)
    print(confusion_matrix(Y_test_array[column], Y_pred.T[column]))

related
              precision    recall  f1-score   support

       False       0.64      0.47      0.54      1246
        True       0.85      0.92      0.88      3990

    accuracy                           0.81      5236
   macro avg       0.74      0.69      0.71      5236
weighted avg       0.80      0.81      0.80      5236

roc auc score 0.6926952211990651
[[ 582  664]
 [ 326 3664]]
request
              precision    recall  f1-score   support

       False       0.88      0.98      0.93      4291
        True       0.81      0.39      0.53       945

    accuracy                           0.87      5236
   macro avg       0.84      0.69      0.73      5236
weighted avg       0.87      0.87      0.86      5236

roc auc score 0.6863383308734042
[[4201   90]
 [ 573  372]]
offer
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      5211
        True       0.00      0.00      0.00        25

    accuracy                           1

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       False       0.96      1.00      0.98      4994
        True       0.72      0.07      0.13       242

    accuracy                           0.96      5236
   macro avg       0.84      0.54      0.56      5236
weighted avg       0.95      0.96      0.94      5236

roc auc score 0.5364892416354171
[[4987    7]
 [ 224   18]]
buildings
              precision    recall  f1-score   support

       False       0.95      1.00      0.97      4947
        True       0.68      0.07      0.12       289

    accuracy                           0.95      5236
   macro avg       0.81      0.53      0.55      5236
weighted avg       0.93      0.95      0.93      5236

roc auc score 0.5319623301109406
[[4938    9]
 [ 270   19]]
electricity
              precision    recall  f1-score   support

       False       0.98      1.00      0.99      5140
        True       0.50      0.04      0.08        96

    accuracy                           0

/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jens/anaconda3/lib/python3.7/si

In [93]:
print('average roc', np.mean(roc_score)) #0.584

average roc 0.5839750112475957


### 9. Export your model as a pickle file

In [96]:
import pickle
outfile = open('MLmodel.pkl', 'wb')
pickle.dump(grid_fit.best_estimator_, outfile)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.